# Import library

In [1]:
import os
import yaml

with open("../conf/service.dev.yaml", 'r') as f:
    configs = yaml.safe_load(f)
os.environ['OPENAI_API_KEY'] = configs['openai']['api_key']

# Create tool with langchain decorator

In [2]:
from langchain.agents import tool

In [3]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Search for weather online
{'query': {'title': 'Query', 'type': 'string'}}


In [5]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [6]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [7]:
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Search for the weather online.
{'query': {'title': 'Query', 'description': 'Thing to search for', 'type': 'string'}}


In [8]:
search.run("sf")

'42f'

# Online temperature retriever tool

## Example: Load temperature

In [9]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [10]:
print(get_current_temperature.name)
print(get_current_temperature.description)
print(get_current_temperature.args)

get_current_temperature
get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.
{'latitude': {'title': 'Latitude', 'description': 'Latitude of the location to fetch weather data for', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'description': 'Longitude of the location to fetch weather data for', 'type': 'number'}}


- OpenAI function 으로 변환 가능

In [11]:
from langchain.tools.render import format_tool_to_openai_function

format_tool_to_openai_function(get_current_temperature)

/home/vscode/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [12]:
get_current_temperature({'latitude': 13, 'longitude': 14})

'The current temperature is 17.3°C'

## Example: Load wikipedia

In [13]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []

    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title} \nSummary: {wiki_page.summary}")
        except (self.wiki_client.exceptions.PageError, self.wiki_client.exceptions.DisambigurationError):
            pass
    
    if not summaries:
        return "No good Wikipedia Search Result was found"
    
    return "\n\n".join(summaries)

In [14]:
print(search_wikipedia.name)
print(search_wikipedia.description)
print(search_wikipedia.args)

search_wikipedia
search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.
{'query': {'title': 'Query', 'type': 'string'}}


In [15]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [16]:
search_wikipedia({'query': "langchain"})

'Page: LangChain \nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: OpenAI \nSummary: OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI Spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the artificial intelligence spring.The organization consists of the non

# Open API spec을 OpenAI function call로 전환하는 방법

In [17]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [18]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [19]:
spec = OpenAPISpec.from_text(text)
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

/home/vscode/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [22]:
model.invoke("What are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "params": {\n    "limit": 3\n  }\n}', 'name': 'listPets'}})

In [23]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "path_params": {\n    "petId": "42"\n  }\n}', 'name': 'showPetById'}})

# Routing
몇 가지 후보군 함수들 중에서 어떤 것을 **선택**해야 할 지 결정하는 방법을 정의

In [24]:
functions = [
    format_tool_to_openai_function(f) for f in (search_wikipedia, get_current_temperature)
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [25]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})

In [26]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "langchain"\n}', 'name': 'search_wikipedia'}})

In [27]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', "You are helpful but sassy assistant"),
    ('user', "{input}")
])

chain = prompt | model

In [28]:
chain.invoke({'input': "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})

- Function call이 수행되면 function call의 argument가 중요하고, 그렇지 않은 경우엔 content가 중요

In [29]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

`OpenAIFunctionsAgentOutputParser`
1. Output의 결과가 function call인지 response인지 체크
2. Function call이라면, 어떤 function 이고 input이 무엇인지 체크 \
아니라면, content를 체크

In [30]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

- Function call example

In [31]:
result = chain.invoke({'input': "what is the weather in sf right now"})

In [32]:
result

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})])

In [33]:
result.tool

'get_current_temperature'

In [34]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [35]:
get_current_temperature(result.tool_input)

'The current temperature is 10.4°C'

- Not function call example

In [36]:
result = chain.invoke({'input': "hi!"})

In [37]:
result

AgentFinish(return_values={'output': 'Hello! How can I assist you today?'}, log='Hello! How can I assist you today?')

In [38]:
result.tool

AttributeError: 'AgentFinish' object has no attribute 'tool'

In [39]:
result.return_values

{'output': 'Hello! How can I assist you today?'}

- Customize routing function

In [40]:
from langchain.schema.agent import AgentFinish

def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            'search_wikipedia': search_wikipedia,
            'get_current_temperature': get_current_temperature
        }
        return tools[result.tool].run(result.tool_input)

In [41]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [42]:
chain.invoke({'input': "What is the weather in san francisco right now"})

'The current temperature is 10.4°C'

In [43]:
result = chain.invoke({'input': "What is langchain?"})

In [44]:
result

'Page: LangChain \nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: OpenAI \nSummary: OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI Spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the artificial intelligence spring.The organization consists of the non

In [45]:
chain.invoke({'input': "hi!"})

'Hello! How can I assist you today?'